In [4]:
import requests
import base64

CLIENT_ID = '22ff7e8460c346588d437905cadfe921'
CLIENT_SECRET = '940cf72983b54061ba8a74317a9817a0'

cc = f"{CLIENT_ID}:{CLIENT_SECRET}"
ccbase64 = base64.b64encode(cc.encode())

url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {ccbase64.decode()}'
}
token = {
    'grant_type': 'client_credentials'
}
response = requests.post(url, data=token, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Success")
else:
    print("Error")
    exit()

Success


In [13]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_playlist(playlist_id,token):
    sp = spotipy.Spotify(auth=token)
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')
    data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']
        if track_id != 'Not available':
            audio_features = sp.audio_features(track_id)[0]
        else:
            audio_features = None
        try:
            if album_id != 'Not available':
                album_info = sp.album(album_id)
            else:
                album_info = None
            if album_info:
                release_date = album_info['release_date']
            else:
                release_date = None
        except:
            release_date = None
        try:
            if track_id != 'Not available':
                track_info = sp.track(track_id)
            else:
                track_info = None
            if track_info:
                popularity = track_info['popularity']
            else:
                popularity = None
        except:
            popularity = None
        try:
            if track_id != 'Not available':
                track_info = sp.track(track_id)
            else:
                track_info = None
            if track_info:
                genre = track_info["genre"]
            else:
                genre = None
        except:
            genre = None
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Genre': genre,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None
        }

        data.append(track_data)

    data = pd.DataFrame(data)
    return data

In [20]:
playlist = '4MXkhoCdT7b3ifglavvhnQ'

data = get_playlist(playlist, access_token)
data.tail()


,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Genre,Release Date,Duration (ms),Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
95,Tie Me Down (with Elley Duhé),"Gryffin, Elley Duhé",Tie Me Down (with Elley Duhé),69t8rpgBN1ov5kCU6LDMuR,4QVS8YCpK71R4FsxSMCjhP,58,None,2018-08-03,218296,0.548,0.839,6,-2.371,1,0.0644,0.13500,0.000000,0.102,0.314,98.932
96,A Sky Full of Stars,Coldplay,Ghost Stories,2G4AUqfwxcV1UdQjm2ouYr,0FDzzruyVECATHXKHFs9eJ,87,None,2014-05-19,267867,0.545,0.675,6,-6.474,1,0.0279,0.00617,0.001970,0.209,0.162,124.970
97,Body Back (feat. Maia Wright),"Gryffin, Maia Wright",Body Back (feat. Maia Wright),3VCPQRcHVxiVaePlQ6TwoF,1z1XegUChLLQfcbtIK8ABn,53,None,2019-10-18,214634,0.686,0.830,8,-4.437,1,0.0369,0.09480,0.000001,0.182,0.486,123.032
98,What If (I Told You I Like You),"Johnny Orlando, kenzie",What If (I Told You I Like You),2zXKlf81VmDHIMtQe3oD0r,4v7DgEMw6EvYyuQ4eWFD9l,62,None,2018-05-18,219187,0.735,0.584,1,-5.047,1,0.1400,0.18900,0.000000,0.125,0.395,93.016
99,Still The One,One Direction,Take Me Home (Expanded Edition),2sWX3HYnZjPZ9MrH6MFsBt,1fZvvlfjgoqjz7HQKEfzyp,57,None,2012-11-12,182133,0.614,0.984,1,-2.419,0,0.0773,0.01180,0.000000,0.650,0.776,142.002


In [15]:
#Since Genre does not exist ('None')
data.drop('Genre', axis=1, inplace=True)
print(data.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
'''
def cosine_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
'''

In [17]:
Scaler = MinMaxScaler()
feature_list = ['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness','Instrumentalness', 'Liveness', 'Valence', 'Tempo']
features = data[feature_list].values
scaled_features = Scaler.fit_transform(features)

In [18]:
def recommendations(input_song, num=5):
    if input_song not in data['Track Name'].values:
        print(f"'{input_song}' not found in the dataset. Please enter a valid song name.")
        return
    
    indexes = data[data['Track Name'] == input_song].index[0]
    similarity = cosine_similarity([scaled_features[indexes]], scaled_features)
    similariy_index = similarity.argsort()[0][::-1][1:num + 1]
    recommendations = data.iloc[ similariy_index][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return recommendations

In [22]:
input_song = "Still The One"
song_recommendations = recommendations(input_song, num=5)
print(f"Recommended songs based on preference for '{input_song}':")
song_recommendations.head()

Recommended songs based on preference for 'Still The One':


,Track Name,Artists,Album Name,Release Date,Popularity
1,On My Way,"Alan Walker, Sabrina Carpenter, Farruko",On My Way,2019-03-21,74
61,Kings & Queens,Ava Max,Heaven & Hell,2020-09-18,78
80,One Time,Justin Bieber,My World,2009-01-01,71
72,2002,Anne-Marie,Speak Your Mind (Deluxe),2018-04-27,78
77,Scared to Be Lonely,"Martin Garrix, Dua Lipa",Scared to Be Lonely,2017-01-27,78


In [26]:
input_song = "Shape Of You"
song_recommendations = recommendations(input_song, num=5)
print(f"Recommended songs based on preference for '{input_song}':")
if song_recommendations is None:
    print("No songs to recommend!")
else:
    song_recommendations.head()
    

'Shape Of You' not found in the dataset. Please enter a valid song name.
Recommended songs based on preference for 'Shape Of You':
No songs to recommend!


In [28]:
input_song = "A Sky Full of Stars"
song_recommendations = recommendations(input_song, num=5)
print(f"Recommended songs based on preference for '{input_song}':")
song_recommendations.head()

Recommended songs based on preference for 'A Sky Full of Stars':


,Track Name,Artists,Album Name,Release Date,Popularity
14,Find U Again (feat. Camila Cabello),"Mark Ronson, Camila Cabello",Find U Again (feat. Camila Cabello),2019-05-30,52
90,Hard 2 Face Reality,"Poo Bear, Justin Bieber, Jay Electronica",Hard 2 Face Reality,2018-04-06,66
97,Body Back (feat. Maia Wright),"Gryffin, Maia Wright",Body Back (feat. Maia Wright),2019-10-18,53
38,Everything,Diamond Eyes,Everything,2018-04-10,0
83,Heading Home,"Alan Walker, Ruben",Heading Home,2020-04-01,54
